In [1]:
import numpy as np
import math

In [2]:
def refraction_angle(i = 0, n_before = 1, n_after = 1):
    return math.asin( (n_before / n_after) * math.sin(i) )

def polar_to_cart(pos, angle):
    grad = math.tan(angle)
    const = pos[1] - (grad * pos[0])
    def f(x):
        return x * grad + const
    return f


def calc_diff(func, x, dx = 1e-10):
    return (func(x+dx) - func(x)) / dx

def calc_intercept(f, g, interval, n_iter = 100, error = 1e-10):
    x_guess = np.mean(interval)
    for i in range(0, n_iter):
        if x_guess < interval[0] or x_guess > interval[1]:
            return x_guess, False
        else:
            
            diff_f_g = f(x_guess) - g(x_guess)
            if abs(diff_f_g) < error:
                return x_guess, True
            
            grad_f_g = calc_diff(f, x_guess) - calc_diff(g, x_guess)
            if abs(grad_f_g) < error:
                print("Failed to converge, gradients too similar")
                return x_guess, False
            
            x_guess = x_guess - (( diff_f_g ) / ( grad_f_g ))
            
    return x_guess, True


def sign_change(x, y):
    if (x > 0 and y > 0) or (x < 0 and y < 0):
        return False
    return True

def estimate_all_intercepts(f, g, interval, n_tests = 100, error = 1e-10):
    x_tests = (np.arange(n_tests, dtype = np.float32) / (n_tests - 1)) * (interval[1] - interval[0]) + interval[0]
    y_vals = np.fromiter((f(xi) - g(xi) for xi in x_tests), np.float32)
    
    test_list = []
    
    for i, val in enumerate(y_vals):
        if(i == 0):
            continue
        
        if sign_change(val, y_vals[i-1]):
            test_list.append([x_tests[i-1], x_tests[i]])
    
    
    # Still try to find 1 intercept
    
    intercepts_tests = []
    
    if len(test_list) == 0:
        intercepts_tests = [calc_intercept(f, g, interval)]
    else:
        for i in test_list:
            intercepts_tests.append(calc_intercept( f, g, i ))
    
    intercepts = []
    for i in intercepts_tests:
        if i[1] == True:
            intercepts.append([i[0], f(i[0])])
    return np.array(intercepts)

def nearest_point(current_pos, possible_pos):
    if len(possible_pos) == 0:
        return None
    elif len(possible_pos) == 1:
        return possible_pos[0]
    else:
        nearest_index = 0
        best_dist = np.inf
        for i, test_pos in enumerate(possible_pos):
            test_dist = np.sum((test_pos - current_pos) ** 2) ** 0.5
            if test_dist < best_dist :
                best_dist = test_dist
                nearest_index = i
        return possible_pos[nearest_index]


def normal_angle(func, x):
    return (math.pi / 2) + math.atan( calc_diff(func, x) )


In [3]:
def curve_1(x):
    return x

def cutoff(x):
    return 2 * x

ans = estimate_all_intercepts(curve_1, cutoff, np.array([-10, 10]))

print(ans)


nearest_ans = nearest_point(np.array([0, 0]), ans)
print(nearest_ans)


angle = normal_angle(curve_1, 10)
print(angle / math.pi)

[[-3.5787142e-20 -3.5787142e-20]]
[-3.5787142e-20 -3.5787142e-20]
0.7500000131685385


In [4]:
refraction_angle(0.1, 1, 2)

0.049937460992958524

In [20]:
class Bound_line:
    def __init__(self, func, interval, is_less_than = True, n = 2):
        self.func = func
        self.interval = interval
        self.is_less_than = is_less_than
        self.n = n
        
    def isin(self, pos):
        print(pos, self.interval)
        if pos[0] < self.interval[0] or pos[0] > self.interval[1]:
            return False
        
        if ( pos[1] <= self.func(pos[0]) ) == self.is_less_than:
            return True
        else:
            return False
    
    def find_intercept(self, ray):
        
        ray_interval = self.interval.copy()
        
        if ray.is_moving_in_pos_x():
            ray_interval[0] = ray.pos[0]
        else:
            ray_interval[1] = ray.pos[0]
        
        all_intercepts =  estimate_all_intercepts( ray.ray_to_cart(), self.func, ray_interval )
        return nearest_point(ray.pos, all_intercepts)
        
    def get_norm_angle(self, x):
        return (math.pi / 2) + math.atan(calc_diff(self.func, x))
        
    
    
class Light_ray:
    def __init__(self, pos, angle):
        self.pos = pos
        self.angle = angle
        
    def ray_to_cart(self):
        return polar_to_cart(self.pos, self.angle)

    def is_moving_in_pos_x(self):
        if (self.angle > math.pi / 2) or (self.angle < -math.pi / 2):
            return False
        else:
            return True
        
    def update_ray(self, bound_line):
        ray_intercept = bound_line.find_intercept(self)
        
        if len(ray_intercept) == 0:
            return None
        
        norm_angle = bound_line.get_norm_angle(self.pos[0])
        
        refracted_angle_rel_line = refraction_angle(norm_angle - self.angle, 1, bound_line.n)
        
        new_angle = (math.pi + norm_angle + refracted_angle_rel_line) % math.pi
        
        return Light_ray(ray_intercept, new_angle)



In [21]:

def curve_3(x):
    return x

ray1 = Light_ray(np.array([0, 1]), - math.pi / 2)

line1 = Bound_line(curve_3, np.array([-10, 10]), True)



In [22]:
new_ray = ray1.update_ray(line1)

In [23]:
print(ray1.pos, ray1.angle / math.pi)
print(new_ray.pos, new_ray.angle / math.pi)

[0 1] -0.5
[6.12323400e-17 1.11022302e-16] 0.6349732719186921


In [ ]:
p